In [ ]:
#!pip install demoji
#!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gzip
import nltk
import spacy
import string
import re
import warnings
import demoji


from nltk.corpus import stopwords
from nltk.collocations import *
from collections import Counter
from nltk.stem import porter
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC







nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('genesis')
nltk.download('averaged_perceptron_tagger')

stop_words = set(stopwords.words('english'))
stemmer = porter.PorterStemmer()

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

nlp = spacy.load("en_core_web_sm")

nltk.download('punkt')
nltk.download('brown')

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package genesis to /root/nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [ ]:
# Importing the dataset
url = 'https://raw.githubusercontent.com/shahrzadko/NLPDigitalMusic/main/Digital_Music_5.json?token=GHSAT0AAAAAAB6LOL42SKHPCCMIVYPOC4REY7AKE2Q'
df_raw = pd.read_json(url)
full_df = pd.DataFrame.from_records(df_raw['data'])

In [ ]:
#creating Sentiment column with overall rating
full_df['Sentiment'] = np.where((full_df['overall'] > 3), 'Positive', 
                          np.where((full_df['overall'] < 3), 'Negative', 'Neutral'))


In [ ]:
#showing that the data is not balanced
full_df['Sentiment'].value_counts() 

Positive    158985
Neutral       6792
Negative      4004
Name: Sentiment, dtype: int64

In [ ]:
#creating  'NewReview'  column using  'reviewText' and 'summary' columns
cols = ['reviewText','summary']
full_df['NewReview'] = full_df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
full_df['NewReview'] = full_df['NewReview'].astype(str)
full_df['review_length'] = full_df['NewReview'].apply(lambda x: len(x.split()))

In [ ]:
music_df_balanced = full_df.drop(columns=['vote','image','reviewTime','style','reviewerName','unixReviewTime','summary','reviewText'])

In [ ]:
#dropping rows if the review is not verified
music_df_balanced = music_df_balanced[music_df_balanced.verified == True]

#dropping rows if the reviewer generally gives very negative reviews
df_filtered = music_df_balanced.groupby('reviewerID').filter(lambda x: not((x['overall'].count() >= 10) and (x['overall'].mean() <= 2)))

#dropping rows if the review is too short
#df_filtered = df_filtered[df_filtered['review_length'] >= 6]


In [ ]:
#checking the values of Sentiment to creat a sample data
df_filtered['Sentiment'].value_counts() 

Positive    141104
Neutral       5396
Negative      2093
Name: Sentiment, dtype: int64

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import wordnet


# Reduce the number of positive sentiment rows to 5000
df_pos = df_filtered[df_filtered['Sentiment'] == 'Positive'].sample(n=10000, random_state=42)

# Split the remaining dataset by sentiment
df_neutral = df_filtered[df_filtered['Sentiment'] == 'Neutral']
df_neg = df_filtered[df_filtered['Sentiment'] == 'Negative']

# Define function for synonym replacement
def synonym_replacement(text):
    words = nltk.word_tokenize(text)
    new_words = []
    for word in words:
        synonyms = []
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if synonyms:
            new_word = synonyms[0]
        else:
            new_word = word
        new_words.append(new_word)
    new_text = ' '.join(new_words)
    return new_text

# Oversample minority classes using synonym replacement
desired_balance = 0.9
while len(df_neutral) / len(df_pos) < desired_balance or len(df_neg) / len(df_pos) < desired_balance:
    if len(df_neutral) / len(df_pos) < desired_balance:
        sample = df_neutral.sample(n=1)['NewReview'].iloc[0]
        new_review = synonym_replacement(sample)
        df_neutral = df_neutral.append({'NewReview': new_review, 'Sentiment': 'Neutral'}, ignore_index=True)
    if len(df_neg) / len(df_pos) < desired_balance:
        sample = df_neg.sample(n=1)['NewReview'].iloc[0]
        new_review = synonym_replacement(sample)
        df_neg = df_neg.append({'NewReview': new_review, 'Sentiment': 'Negative'}, ignore_index=True)




# Combine oversampled subsets back into a single dataframe
df_balanced = pd.concat([df_pos, df_neutral, df_neg], ignore_index=True)

# Shuffle the dataframe
df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)


In [ ]:
df_balanced['Sentiment'].value_counts() 

Positive    10000
Negative     9000
Neutral      9000
Name: Sentiment, dtype: int64

In [ ]:
#creating balanced data using equal number of each sentiment
#sample_size = min(df_filtered['Sentiment'].value_counts())
#df_balanced = df_filtered.groupby('Sentiment').apply(lambda x: x.sample(sample_size)).reset_index(drop=True)

In [ ]:
#df_balanced['Sentiment'].value_counts() 

In [ ]:
df_balanced.head()

,overall,verified,reviewerID,asin,Sentiment,NewReview,review_length
0,5.0,True,ALIQSYVGG972W,B0064Y4A0W,Positive,Love! Five Stars,3.0
1,NaN,NaN,NaN,NaN,Negative,every time iodine attempt to bargain angstrom ...,NaN
2,3.0,True,A9FS0PG49LNG5,B004YFAZT8,Neutral,I enjoy hearing music and listed to all sorts ...,13.0
3,NaN,NaN,NaN,NaN,Negative,Department_of_Energy n't expression to rich_pe...,NaN
4,NaN,NaN,NaN,NaN,Negative,iodine give this download alone one star becau...,NaN


In [ ]:
#drop non-English comments
from langdetect import detect

def is_english(text):
    try:
        lang = detect(text)
        return lang == 'en'
    except:
        return False

df_balanced = df_balanced[df_balanced['NewReview'].apply(is_english)]


In [ ]:
#checking the values of Sentiment after removing non-English reviews
df_balanced['Sentiment'].value_counts() 

Positive    9449
Neutral     8744
Negative    8170
Name: Sentiment, dtype: int64

In [ ]:
#df_balanced = df_balanced.sample(n=1000, random_state=42)

In [ ]:
df_balanced = df_balanced.drop(columns=['verified','overall','asin','reviewerID'])

In [ ]:
df_balanced.head()

,Sentiment,NewReview,review_length
0,Positive,Love! Five Stars,3.0
1,Negative,every time iodine attempt to bargain angstrom ...,NaN
2,Neutral,I enjoy hearing music and listed to all sorts ...,13.0
3,Negative,Department_of_Energy n't expression to rich_pe...,NaN
4,Negative,iodine give this download alone one star becau...,NaN


In [ ]:
# import re
# import string
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer, WordNetLemmatizer

# # Initialize stopwords
# STOPWORDS = set(stopwords.words('english'))

# # Convert NewReview to lowercase
# df_balanced['NewReview'] = df_balanced['NewReview'].str.lower()

# # Remove URLs from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: re.sub(r'http\S+', '', x))

# # Remove currency symbols from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: re.sub(r'£|\$', '', x))

# # Remove phone numbers from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: re.sub(r'^[\(]?[\d]{3}[\)\-]?[\s\.]?[\d]{3}[\-\s\.]?[\d]{4}$', '', x))

# # Remove digits from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: re.sub(r'\d+', '', x))

# # Remove email addresses from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: re.sub(r'^.+@[^\.].*\.[a-z]{2,}$', '', x))

# # Remove special characters from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: re.sub(r'[\'“’]', '', x))

# # Remove stopwords from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: " ".join([word for word in x.split() if word.lower() not in STOPWORDS]))

# # Remove empty values from NewReview
# df_balanced['NewReview'] = df_balanced['NewReview'].apply(lambda x: '' if pd.isna(x) or not isinstance(x, str) else x)


In [ ]:
from nltk.stem import *

# Initialize stopwords
STOPWORDS = set(stopwords.words('english'))
# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Define function for text cleaning
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove currency symbols
    text = re.sub(r'£|\$', '', text)

    # Remove phone numbers
    text = re.sub(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', '', text)

    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)

    # Remove punctuation and special characters except emojis
    #text = re.sub(r'[^\w\s' + emoji.get_emoji_regexp() + ']', '', text)
    text = demoji.replace(text, '')

    # Tokenize text into words
    words = nltk.word_tokenize(text)
    
    # Remove stopwords and handle negation
    negation_words = ['not', 'no', 'never', 'neither', 'nor']
    words = [word if word.lower() in negation_words else word.lower() for word in words]
    words = [word for word in words if word.lower() not in STOPWORDS]
    words = [f'NOT_{words[i+1]}' if (i < len(words)-1 and words[i].lower() in negation_words and words[i+1] not in string.punctuation) else words[i] for i in range(len(words))]
    
    # Lemmatize words
    words = [lemmatizer.lemmatize(word) for word in words]

    # Stem words
    words = [stemmer.stem(word) for word in words]

    # Join words back into text
    cleaned_text = ' '.join(words)
    return cleaned_text


In [ ]:
#df_balanced['processed_review'] = df_balanced['NewReview'].apply(clean_text)
from multiprocessing import cpu_count, Pool
pool = Pool(cpu_count())
tasks = df_balanced['NewReview'].tolist()
done = pool.map(clean_text, tasks)  # parallel processing
df_balanced['processed_review'] = done

In [ ]:
df_balanced.head()

,Sentiment,NewReview,review_length,processed_review
0,Positive,Love! Five Stars,3.0,love ! five star
1,Negative,every time iodine attempt to bargain angstrom ...,NaN,everi time iodin attempt bargain angstrom new ...
2,Neutral,I enjoy hearing music and listed to all sorts ...,13.0,enjoy hear music list sort style . three star
3,Negative,Department_of_Energy n't expression to rich_pe...,NaN,department_of_energi n't express rich_person d...
4,Negative,iodine give this download alone one star becau...,NaN,"iodin give download alon one star , even thoug..."


In [ ]:
df_balanced = df_balanced.drop(columns=['NewReview'])

In [ ]:
df_balanced.head()

,Sentiment,review_length,processed_review
0,Positive,3.0,love ! five star
1,Negative,NaN,everi time iodin attempt bargain angstrom new ...
2,Neutral,13.0,enjoy hear music list sort style . three star
3,Negative,NaN,department_of_energi n't express rich_person d...
4,Negative,NaN,"iodin give download alon one star , even thoug..."


In [ ]:
X = pd.DataFrame(df_balanced['processed_review'])
y = df_balanced['Sentiment']

TfidfVectorizer / Naive Bayes

In [ ]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")


Accuracy: 0.849


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[2132  190  129]
 [ 367 1920  336]
 [  30  141 2664]]
              precision    recall  f1-score   support

    Negative       0.84      0.87      0.86      2451
     Neutral       0.85      0.73      0.79      2623
    Positive       0.85      0.94      0.89      2835

    accuracy                           0.85      7909
   macro avg       0.85      0.85      0.85      7909
weighted avg       0.85      0.85      0.85      7909



Count Vectorizer / Naive Bayse

In [ ]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 
# Define CountVectorizer
count_vectorizer = CountVectorizer()

# Define Naive Bayes classifier
nb_classifier = MultinomialNB()

# Combine CountVectorizer and Naive Bayes classifier into a pipeline
pipeline = Pipeline([
    ('vectorizer', count_vectorizer),
    ('classifier', nb_classifier)
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Make predictions on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")


Accuracy: 0.8305727652041978


In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[2004  260  187]
 [ 344 1861  418]
 [  16  115 2704]]
              precision    recall  f1-score   support

    Negative       0.85      0.82      0.83      2451
     Neutral       0.83      0.71      0.77      2623
    Positive       0.82      0.95      0.88      2835

    accuracy                           0.83      7909
   macro avg       0.83      0.83      0.83      7909
weighted avg       0.83      0.83      0.83      7909



#### Logistic Regression

In [ ]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 
# Define the pipeline
model = make_pipeline(
    CountVectorizer(),
    StandardScaler(with_mean=False),
    LogisticRegression(random_state=42)
)

# Fit the model to the training data
model.fit(X_train['processed_review'], y_train)

# Predict the sentiment of the test data
y_pred = model.predict(X_test['processed_review'])

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8743203944872929


In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[2283   94   74]
 [ 149 2203  271]
 [  79  327 2429]]
              precision    recall  f1-score   support

    Negative       0.91      0.93      0.92      2451
     Neutral       0.84      0.84      0.84      2623
    Positive       0.88      0.86      0.87      2835

    accuracy                           0.87      7909
   macro avg       0.87      0.88      0.88      7909
weighted avg       0.87      0.87      0.87      7909



#### Gradient Boosting

In [ ]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 
# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('gb', GradientBoostingClassifier())
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")


Accuracy: 0.831


In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[2087  163  201]
 [ 352 1893  378]
 [  23  218 2594]]
              precision    recall  f1-score   support

    Negative       0.85      0.85      0.85      2451
     Neutral       0.83      0.72      0.77      2623
    Positive       0.82      0.91      0.86      2835

    accuracy                           0.83      7909
   macro avg       0.83      0.83      0.83      7909
weighted avg       0.83      0.83      0.83      7909



SVM

In [ ]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 
# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC())
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")


Accuracy: 0.916


In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[2278   94   79]
 [  84 2278  261]
 [  15  134 2686]]
              precision    recall  f1-score   support

    Negative       0.96      0.93      0.94      2451
     Neutral       0.91      0.87      0.89      2623
    Positive       0.89      0.95      0.92      2835

    accuracy                           0.92      7909
   macro avg       0.92      0.92      0.92      7909
weighted avg       0.92      0.92      0.92      7909



In [ ]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 

# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC())
])


# Define the hyperparameter grid
param_grid = {
    'svm__C': [0.1, 1, 10, 100,200],
    'svm__kernel': ['linear', 'rbf', 'sigmoid','precomputed','poly'],
    'svm__gamma': ['scale', 'auto','float','array-like']
}


# Define the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the pipeline on the training data
grid_search.fit(X_train['processed_review'], y_train)

# Print the best hyperparameters
print("Best parameters: ", grid_search.best_params_)

# Predict on the test data using the best estimator
y_pred = grid_search.best_estimator_.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")


In [ ]:
# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
 
# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC(C= 100, gamma= 'scale', kernel='rbf'))
])

# Fit the pipeline on the training data
pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")

Accuracy: 0.921


In [ ]:
from sklearn.calibration import CalibratedClassifierCV

# Split the data into training and testing sets with stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC(C= 100, gamma= 'scale', kernel='rbf', probability=True)) # set probability to True
])

# Wrap the pipeline in a calibrated classifier
calibrated_pipeline = CalibratedClassifierCV(pipeline, method='sigmoid', cv=5) # you can choose different calibration methods

# Fit the pipeline on the training data
calibrated_pipeline.fit(X_train['processed_review'], y_train)

# Predict on the test data
y_pred = calibrated_pipeline.predict(X_test['processed_review'])

# Calculate the accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.3f}")
